In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title pre-processing
import pandas as pd
from copy import deepcopy
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
import math
# dataset_path=r'test.csv'
dataset_path=r'/content/drive/MyDrive/diabetes.csv'
binaryColumns=['Diabetes_binary','HighBP','High_Cholesterol','Cholesterol_Check','Smoker','Stroke','HeartDiseaseorAttack','Physical_Activity','Fruits','Veggies','Heavy_Alcohol_Consumption','Any_Health_Care','No_Doctor_because_of_Cost','Difficulty_Walking']
intColumns=['Physical_Health','Mental_Health']
df = pd.read_csv(dataset_path, index_col=0)
df.columns=[col.replace(" ","_") for col in df.columns]
modes=df.mode()
legalIncomeValues=['Cat'+str(i) for i in range(1,9)]
legalEducationValues=['Cat'+str(i) for i in range(1,7)]
legalGeneralHealthValues=['High','Medium','Low','Very Low','Good']
legalSexValues=['male','female']
legalBinaryValues=[0,1]
legalIntValues=[i for i in range(0,31)]
legalValuesDict={'Income':legalIncomeValues,'Education':legalEducationValues,'General_Health':legalGeneralHealthValues,'Sex':legalSexValues}
ImputerDict=dict()
scalarDict=dict()

def checkNumberIsSmaller(min,number):
    try:
        if min < number <math.inf :return True
    except:
        return False

def bmiBin(x):
    if x<18.5:return 0
    elif 18.5 <=x<25:return 1
    elif  25 <=x<30:return 2
    else:return 3

def ageBin(x):
    if x<15:return 0
    elif 15<=x<25:return 1
    elif 25<=x<64:return 2
    else:return 3


print(df.shape)
for i in range(df.shape[0]):
    nullFlag = 0

    for col in df.columns:
        temp = deepcopy(df.loc[i][col])
        if col in legalValuesDict.keys() or col in binaryColumns or col in intColumns:
            if (col in legalValuesDict.keys() and df.loc[i][col] not in legalValuesDict[col]) or (col in binaryColumns and  df.loc[i][col] not in legalBinaryValues) or (col in intColumns and df.loc[i][col] not in legalIntValues):
             nullFlag+=1
             df.at[i, col]=modes.loc[0][col]
             print()

        elif not checkNumberIsSmaller(0, df.loc[i][col]):
         nullFlag += 1
         df.at[i, col]=pd.NA

    if nullFlag>np.ceil(df.shape[1]/2):
        print()
        df=df.drop(i)

binFuncs={'BMI':bmiBin,'Age':ageBin}
numericColumns=[]
for col in df.columns:
    if (col not in legalValuesDict.keys()) and (col not in binaryColumns) and (col not in intColumns):
     df.loc[:,col].fillna( df.loc[:,col].mean(), inplace=True)
     df.loc[:, col] = df.loc[:,col].apply(binFuncs[col])
     numericColumns.append(col)

# df[numericColumns] = StandardScaler().fit_transform(df[numericColumns])
print()


for col in intColumns:
 est = KBinsDiscretizer(n_bins=5, encode='onehot', strategy='uniform')
 transformed = est.fit_transform(df[[col]])
 df=df.drop([col], axis=1)
 transformed=np.transpose(np.asarray(transformed.toarray())).tolist()
 print()
 for i in range(len(transformed)):
  df.insert(i+4, column=col+str(i), value=transformed[i], allow_duplicates=True)


oneHotEncoderColumns=list(legalValuesDict.keys())+numericColumns

for col in oneHotEncoderColumns:
 ohe=OneHotEncoder(categories='auto')
 transformed = ohe.fit_transform(df[[col]])
 df=df.drop([col], axis=1)
 transformed=np.transpose(np.asarray(transformed.toarray())).tolist()
 print()
 for i in range(len(transformed)):
  df.insert(i+4, column=col+str(i), value=transformed[i], allow_duplicates=True)

# df.to_csv('test2.csv')
df.to_csv('/content/drive/MyDrive/diabetesPreprocessed.csv')


<div>
Here we perform the preprocessing step and remove invalid values in the data set and also encode the categorical data with oneHotEncoder. To check the validity of binary, categorical and integer data, we check whether their values are in the ranges of valid values or not, and to check numerical data such as bmi and age, we check that the value is greater than zero. and smaller than infinity (because age and bmi are zero or infinity completely meaningless) and also if their type is numeric and finally we normalize numerical and integer data using binning as follows We put bmi values in four categories: underweight, healthy weight, overweight and obese, and we put age values in four categories: Children, Youth, Adults, Seniors, and Physical and Mental health values in five uniform categories (because General_Health values are also in five are categories). Also, if the attribute values of binary, categorical and integer data are equal to null or an illegal value, we replace it with the mode of that data, and if the data is numerical, we replace it with its average.
</div>

In [23]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from copy import deepcopy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

In [24]:
def my_roc_auc_score(model, X, y):
    confusionMatrixDict["lr:"+str(model.learning_rate)+" "+"md:"+str(model.max_depth)+" "+"n:"+str(model.n_estimators)+" "+"cb:"+str(model.colsample_bytree)]=confusionMatrix(y,model.predict(X))
    # if np.sum(y)==0:return 0
    return roc_auc_score(y, model.predict_proba(X)[:,1])

def confusionMatrix(test_y,pred_y):
    # test_y=test_y.to_numpy()
    tn, fp, fn, tp=confusion_matrix(test_y, pred_y).ravel()
    d = {'positive': [tp, fp], 'negative': [fn, tn]}
    # the rows represent the predicted values
    df = pd.DataFrame(data=d)
    return df


With the confusion Matrix function, we generate the confusion matrix, in which the confusion matrix available in sklearn is used for faster speed, but a function has also been written that gives an equivalent answer without the usage of sklearn.

In [ ]:
# def confusionMatrix(test_y,pred_y):
#     # test_y=test_y.to_numpy()
#     tp = 0
#     fn = 0
#     fp = 0
#     tn = 0
#     print()
#     for i in range(len(pred_y)):
#      if pred_y[i] == test_y[i]:
#          if pred_y[i] == 0:
#              tn += 1
#          elif pred_y[i] == 1:
#              tp += 1
#      else:
#          if pred_y[i] == 0:
#              fn += 1
#          elif pred_y[i] == 1:
#              fp += 1

#     d = {'positive': [tp, fp], 'negative': [fn, tn]}
#     # the rows represent the predicted values
#     df = pd.DataFrame(data=d)
#     return df

In [26]:
dataset_path=r'/content/gdrive/MyDrive/diabetesPreprocessed.csv'
df = pd.read_csv(dataset_path, index_col=0)
model = XGBClassifier(Learning_rate=0.1,Max_depth=4,N_estimator=200,Subsample=0.5,Colsample_bytree=1,Random_seed=123,Eval_metric='auc',Verbosity=1,early_stopping_rounds=10)
y=deepcopy(df.loc[:,'Diabetes_binary'])
df=df.drop(['Diabetes_binary'], axis=1)
confusionMatrixDict=dict()
X_train, X_test, y_train, y_test =train_test_split(df, y, test_size=0.2)
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)])
y_pred = model.predict(X_test)
modelName="learning_rate:"+str(model.learning_rate)+" "+"max_depth:"+str(model.max_depth)+" "+"n_estimators:"+str(model.n_estimators)+" "+"colsample_bytree:"+str(model.colsample_bytree)
print("confusion matrix of model with specifications such as "+modelName)
cm=confusionMatrix(y_test.to_numpy(),y_pred)
print(cm)
accuracy = (cm.iloc[0][0] + cm.iloc[1][1]) / (cm.iloc[0][0] + cm.iloc[1][1] + cm.iloc[0][1] + cm.iloc[1][0])
precision = cm.iloc[0][0] / (cm.iloc[0][0] + cm.iloc[1][0])
recall = cm.iloc[0][0] / (cm.iloc[0][0] + cm.iloc[0][1])
print("accuracy,precision and recall of model with specifications such as "+modelName+" are:")
print("accuracy:"+str(accuracy))
print("precision:"+str(precision))
print("recall:"+str(recall))



[0]	validation_0-error:0.291325	validation_1-error:0.284179
[1]	validation_0-error:0.283704	validation_1-error:0.27463
[2]	validation_0-error:0.284358	validation_1-error:0.276045
[3]	validation_0-error:0.279283	validation_1-error:0.272226
[4]	validation_0-error:0.279088	validation_1-error:0.271801
[5]	validation_0-error:0.279283	validation_1-error:0.272226
[6]	validation_0-error:0.283261	validation_1-error:0.274489
[7]	validation_0-error:0.283067	validation_1-error:0.274418
[8]	validation_0-error:0.282218	validation_1-error:0.273782
[9]	validation_0-error:0.273748	validation_1-error:0.267911
[10]	validation_0-error:0.27366	validation_1-error:0.267911
[11]	validation_0-error:0.271608	validation_1-error:0.265931
[12]	validation_0-error:0.272086	validation_1-error:0.266355
[13]	validation_0-error:0.270742	validation_1-error:0.265295
[14]	validation_0-error:0.267453	validation_1-error:0.262041
[15]	validation_0-error:0.267241	validation_1-error:0.262183
[16]	validation_0-error:0.26694	vali

<div>
Here, according to the specifications mentioned in the XGBClassifier project, we determine the test and training data in such a way that the test data is 20% of the total data and the rest of the data is used to train the model. Finally, we calculate precision, accuracy and recall using the confusion matrix. It should be noted that the positive and negative columns correspond to the predictor answers and the rows 0 and 1 correspond to the positive and negative labels, respectively.
</div>

In [7]:
parameters={
'learning_rate': [0.02, 0.05, 0.1, 0.3],
'max_depth' : [2, 3, 4],
'n_estimators': [100, 200, 300],
'colsample_bytree' : [0.8, 1],
'Subsample':[0.5],
'Eval_metric':['auc']
}
model2 = XGBClassifier()
clf = GridSearchCV(model2, parameters,
                   cv=StratifiedKFold(n_splits=3,shuffle=True),
                   scoring=my_roc_auc_score)
clf.fit(df.to_numpy(),y.to_numpy())
print("best parameters are")
print(clf.best_params_ )
accuracyDict=dict()
recallDict=dict()
precisionDict=dict()
for key in confusionMatrixDict.keys():
    df=confusionMatrixDict[key]
    accuracyDict[key] = (df.iloc[0][0] + df.iloc[1][1]) / (df.iloc[0][0] + df.iloc[1][1] + df.iloc[0][1] + df.iloc[1][0])
    precisionDict[key] = df.iloc[0][0] / (df.iloc[0][0] + df.iloc[1][0])
    recallDict[key] = df.iloc[0][0] / (df.iloc[0][0] + df.iloc[0][1])
    print("confusion matrix of model with specifications such as "+key)
    print(df)


best parameters are
{'Eval_metric': 'auc', 'Subsample': 0.5, 'colsample_bytree': 0.8, 'learning_rate': 0.3, 'max_depth': 2, 'n_estimators': 300}
confusion matrix of model with specifications such as lr:0.02 md:2 n:100 cb:0.8
   positive  negative
0      8974      2808
1      3472      8309
confusion matrix of model with specifications such as lr:0.02 md:2 n:200 cb:0.8
   positive  negative
0      9073      2709
1      3336      8445
confusion matrix of model with specifications such as lr:0.02 md:2 n:300 cb:0.8
   positive  negative
0      9096      2686
1      3254      8527
confusion matrix of model with specifications such as lr:0.02 md:3 n:100 cb:0.8
   positive  negative
0      9336      2446
1      3685      8096
confusion matrix of model with specifications such as lr:0.02 md:3 n:200 cb:0.8
   positive  negative
0      9289      2493
1      3487      8294
confusion matrix of model with specifications such as lr:0.02 md:3 n:300 cb:0.8
   positive  negative
0      9253      2529
1

<div>
Here we use different values for hyper parameters to see which model and with what hyper parameters values has the best performance according to my_roc_auc_score function. It should be noted that in this function we also calculate the confusion matrices of different models in order to finally print them and also use them to calculate the precision, accuracy and recall of the model.
</div>

In [ ]:
import json
evaluationDict={'accuracy':accuracyDict,'precision':precisionDict,'recall':recallDict}
with open('/content/gdrive/MyDrive/data.json', 'w') as fp:
 json.dump(evaluationDict, fp)


In [1]:

from matplotlib import pyplot as plt
evaluationDict=None
with open('/content/gdrive/MyDrive/data.json') as json_file:
    evaluationDict = json.load(json_file)

Mounted at /content/gdrive


In [20]:

plt.rcParams["figure.figsize"] = (200,100)
for key in evaluationDict.keys():
 x = list(evaluationDict[key].keys())
 y = list(evaluationDict[key].values())
 print(y)
 print()
 plt.xlabel('models')
 print(key)
 plt.ylabel(key)
 plt.bar(x, y, color='maroon', width=0.4)
 plt.show()


[0.7334804566481348, 0.7434537198149641, 0.747909858676739, 0.7398039298900819, 0.7462122819674915, 0.7475279039171583, 0.7432415227263082, 0.7472732674107712, 0.7482493740185885, 0.7465942367270721, 0.7489708441200187, 0.7481220557653949, 0.7477825404235454, 0.7486313287781692, 0.7490132835377499, 0.7479947375122014, 0.7475279039171583, 0.74723082799304, 0.7481644951831261, 0.7486313287781692, 0.7490132835377499, 0.7479947375122014, 0.7487586470313627, 0.7491830412086746, 0.7481220557653949, 0.74723082799304, 0.7471883885753088, 0.7482918134363197, 0.7481644951831261, 0.7488859652845563, 0.7487586470313627, 0.7469337520689217, 0.7454908118660611, 0.7470610703221152, 0.7445147052582438, 0.7432839621440394, 0.7346263209268769, 0.7433264015617705, 0.7476127827526207, 0.7412044306752111, 0.7455756907015236, 0.7471459491575776, 0.7426049314603403, 0.7473581462462335, 0.7477825404235454, 0.7451937359419428, 0.7478674192590078, 0.7477825404235454, 0.747018630904384, 0.7476127827526207, 0.748

[0.7210348706411699, 0.7311628656620195, 0.7365182186234818, 0.7169956224560325, 0.7270663744520977, 0.7301349325337332, 0.7221705426356589, 0.7268904290943073, 0.7295918367346939, 0.7332049121117264, 0.7376265694613203, 0.7372393086099164, 0.7301900181345108, 0.7337056242520942, 0.7342702012136697, 0.7289968652037617, 0.7299897484425518, 0.7301675094816688, 0.7380332139368284, 0.739514348785872, 0.7386141834743006, 0.7320889594916601, 0.7332484481935381, 0.7332009531374106, 0.730723699786915, 0.7307875138646808, 0.7308027577462557, 0.7366717903082275, 0.7348217153870864, 0.734431209529139, 0.7331742243436754, 0.7294377415030361, 0.7279892803657287, 0.7301913648584533, 0.7273516414141414, 0.726439687179082, 0.7204018177469504, 0.7314336454633517, 0.7361774467740629, 0.7221701063164477, 0.7274585331341875, 0.7316995544239338, 0.7214346579905492, 0.7279624560031287, 0.7287471597586774, 0.7337756918595242, 0.737806367559645, 0.7373770225221563, 0.7292273765456407, 0.7320079522862823, 0.73

[0.7616703445934476, 0.7700729927007299, 0.7720251230690884, 0.7923951790867425, 0.7884060431166185, 0.7853505347139704, 0.7906976744186046, 0.7922254286199287, 0.7889152945170599, 0.7753352571719572, 0.7728738754031573, 0.7710914955016126, 0.7860295365812257, 0.7805975216431845, 0.7805126464097776, 0.7895094211509082, 0.785690035647598, 0.7843320319130878, 0.7694788660668817, 0.7676964861653369, 0.770836869801392, 0.7822950263113223, 0.7820404006111017, 0.7834832795790189, 0.7858597861144118, 0.7828891529451706, 0.7827194024783568, 0.7728738754031573, 0.7766083856730606, 0.7797487693091156, 0.7822101510779155, 0.7850959090137498, 0.7839076557460533, 0.7837379052792395, 0.7822950263113223, 0.7805126464097776, 0.766932609064675, 0.7690544898998473, 0.7718553726022747, 0.784077406212867, 0.7854354099473774, 0.7805126464097776, 0.790443048718384, 0.7899337973179427, 0.7894245459175012, 0.7696486165336954, 0.7690544898998473, 0.7697334917671024, 0.7858597861144118, 0.7812765235104396, 0.78

<div >
In general, it can be seen that different values for hyper parameters do not cause significant changes in precision, accuracy and recall,and the confusion matrices of different models are also similar.
</div>